In [1]:
# Initialization
import torch
from dotenv import load_dotenv
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from functools import lru_cache
import pandas as pd
from collections import Counter
import os
import random

"""Load environment variables and configure device."""
load_dotenv("keys.txt")
hf_token = os.getenv("HF_TOKEN")
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
if torch.cuda.is_available():
    print(f"Using GPU: {torch.cuda.get_device_name()}")
else:
    print(f"CUDA not found")

/home/erdi/Documents/temp/temp_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using GPU: NVIDIA GeForce RTX 3080 Laptop GPU


In [2]:
from rag_main import (load_pdfs_from_directory, convert_pdfs_chunks, 
create_and_save_vector_store, load_vector_store, retrieve_context,
load_generation_model, generate_answer, format_references_and_pages)

Using GPU: NVIDIA GeForce RTX 3080 Laptop GPU


# Processing and Chunking PDFs

- In this section, our goal is to process all the pdf files and create a vector store that can be used for the downstreaam retrival tasks. To do so, we want to respect the general structure. For example, we need to make sure we dont mix up one document with another during chunking.
- Note that the following approach works well if the documents are mostly composed of text since the approach here is adopted for text-based retrival not a multi-model approach. This will be the topic for another project soon.

In [3]:
data_path = "cbo_documents/"

In [4]:
pdf_documents = load_pdfs_from_directory(data_path)

Loading: 60115-MBR.pdf
Loading: 60479-MBR.pdf
Loading: 59822_MBR.pdf
Loading: 60193-MBR.pdf
Loading: 60592-MBR.pdf
Loading: 59973-MBR.pdf
Loading: 60843-MBR.pdf
Loaded 7 PDFs from budgets/


- *pdf_documents* is a dictory such that keys are the indivual pdf names and values are Langchain "Document" objects.
- We can make a quick look to check if the loader gets the right content from each file.

In [5]:
# check the pdf names
print(f"uploded files: {pdf_documents.keys()}")

# lets see what is in one of those pdfs
doc = pdf_documents['60115-MBR.pdf'][0]
print(doc.metadata)

# we can now look at some of the stuff in this content
print(doc.page_content[:100])

uploded files: dict_keys(['60115-MBR.pdf', '60479-MBR.pdf', '59822_MBR.pdf', '60193-MBR.pdf', '60592-MBR.pdf', '59973-MBR.pdf', '60843-MBR.pdf'])
{'source': 'budgets/60115-MBR.pdf', 'page': 0}
 
The amounts shown in this report include the surplus or deficit in the Social Security trust funds


Now that we have this function, we will convert the extracted text into chunks by making sure that each chunk is tied to its respective file. We will achive this by adding the source information to the metadata of each chunk. Then we will inspect some of the chunks to see if the content is correct. 

In [6]:
chunks = convert_pdfs_chunks(pdf_documents, chunk_size=500, chunk_overlap=50)

Splitting pages from 60115-MBR.pdf into chunks...
Splitting pages from 60479-MBR.pdf into chunks...
Splitting pages from 59822_MBR.pdf into chunks...
Splitting pages from 60193-MBR.pdf into chunks...
Splitting pages from 60592-MBR.pdf into chunks...
Splitting pages from 59973-MBR.pdf into chunks...
Splitting pages from 60843-MBR.pdf into chunks...
Total chunks created: 312


In [7]:
# let pick a random chunks and inspect its content
random_chunk = random.choice(chunks)
print(f"source_pdf: {random_chunk.metadata['source']}")
print(f"source_page: {random_chunk.metadata['page']}")
print(f"chunk_content:\n{random_chunk.page_content[:400]}")

source_pdf: 60193-MBR.pdf
source_page: 6
chunk_content:
surplus in April. This year, that surplus was $208 billion, CBO estimates—$32 billion more than 
the amount recorded last April. Revenues and outlays were higher than they were a year ago. 
Outlays in April 2023 were lower than they otherwise would have been because certain federal 
payments due on April 1, 2023, a Saturday, were made in March. If not for that shift, the surplus 
in April 2024 wou


# Create Vector Store From Chunks
- We are ready to create a vector store. To do that, we need an embedding model that can convert chunks into a high dimensional vector, this is simply another neural network as you can guess. Bottom line is that these models are trained to map similar context to similar vectors(in term of some metric such as cosine similarity). Key point is that it is a good idea to pick a task-spesific embedding model. For example, embedding models such as *FinBERT and FinLang* are primariluty trained on financial documents. We can also use *Sentence-T5 or MiniLM-L6-v2* which are good for general-purpose embeddings. For example, at the bottom of this notebook, you can observe that *FinLang* does a way better job then *Sentence-T5* since we demonstrate the model in financial documents.
  
- To store our embeddings(vectors plus some metadata), we will use FAISS (Facebook AI Similarity Search) framework. Despite the fact that limited functionality for dynamics updates or RAM usage, it is a good starting point for a small project. The following function will create two files; *index.faiss* which contains the actual vector embeddings and *index.pkl* which has metadata associated with each embedding.

In [8]:
embedding_model_name = 'sentence-transformers/sentence-t5-base'
save_path = "vectorstore.faiss"
vectorstore = create_and_save_vector_store(chunks,embedding_model_name,save_path)

Generating embeddings for chunks...
Vector store saved to vectorstore.faiss


# Retrive Information Based on Query
We are now ready to retrive information from our vector store based on our queries. Note that we spesify how many chunks we would like to retrive but did not implement any *ranking logic* within this function. This idea will come up in the next section

In [9]:
vectorstore = load_vector_store(embedding_model_name)

# Step 2: Query the vector store
query = "What is offical US policy on Pandemics and Biodefense"
max_chunks = 2
retrieved_chunks, source_info = retrieve_context(vectorstore, query, max_chunks)

# Step 3: Display retrieved chunks
for i, chunk in enumerate(retrieved_chunks):
    print(f"Chunk {i + 1}: {chunk.page_content[:500]}...\n")
    # source_pdf:[number_of_chunks, pages]
    print(f"source information: {source_info}")
    print("==========================================")

Loading vector store from vectorstore.faiss...
Querying vector store for: 'What is offical US policy on Pandemics and Biodefense'
Chunk 1: MONTHLY BUDGET REVIEW FOR APRIL 2024  MAY 8, 2024 
5 
 Medicaid outlays decreased by $3 billion (or 1 percent) as states continue to reassess the 
eligibility of enrollees who remained in the program for the duration of the coronavirus 
public health emergency. (The continuous-enrollment requirement ended on 
March 31, 2023.) 
Outlays increased substantially in several other areas: 
 Spending by the Department of Defense (DoD) was $36 billion (or 8 percent) greater than...

source information: {'60193-MBR.pdf': [1, [5]], '60115-MBR.pdf': [1, [7]]}
Chunk 2: because in March 2023, the department recorded costs associated with extending the pause 
on student loan repayments that was instituted during the pandemic. 
 Outlays for Medicaid decreased by $8 billion (or 12 percent). 
 Spending by DoD decreased by $6 billion (or 8 percent). 
 Outlays related 

# Generation Model

- The last step is to spesify which LLM we will use to process the retrived information and give us a an organized final answer. Of course, there are 100s options. We would like to use an open-source one, lets pick *Llama-2-7b-chat*. It is a relatively light-weight model. We will load quantized version of it to speed up the inference.
- Note that we have a relative simple logic there to recreate or use the exisiting vector store. We would like to update it when new files are added or the existing ones are modified. In a business setting, this process has to be managed by careful reindexing but we dont need to do that at this point

In [10]:
# generation_model = 'meta-llama/Llama-2-7b-chat-hf'
# tokenizer, model = load_generation_model(generation_model)
# max_new_tokens = 200
# temperature = 0.7

# answer = generate_answer(tokenizer, model, retrieved_chunks, query, max_new_tokens,temperature)
# print(answer)

In [11]:
if __name__ == "__main__":
    # simple logic to recreate or use the existing vector database
    UPDATE_VS = True
    chunk_size = 500
    chunk_overlap = 50
    data_path = "budgets/"
    vectorstore_path = "vectorstore.faiss"
    embedding_model_name = 'FinLang/finance-embeddings-investopedia'
    generation_model = 'meta-llama/Llama-2-7b-chat-hf'
    
    max_chunks = 20
    max_new_tokens = 300
    temperature = 0.7
    show_source = True

    # User query
    query = ("What was the primary reason for the $309 billion increase in outlays by the Department of Education in fiscal year 2024?")
    
    if UPDATE_VS:
        pdf_documents = load_pdfs_from_directory(data_path)
        chunks = convert_pdfs_chunks(pdf_documents, chunk_size, chunk_overlap)
        vectorstore = create_and_save_vector_store(chunks,embedding_model_name,vectorstore_path)
        
    
    # Load vector store
    vectorstore = load_vector_store(embedding_model_name,vectorstore_path)

    # Retrieve relevant chunks
    retrieved_chunks,_ = retrieve_context(vectorstore, query, max_chunks,show_source)

    # Load generation model
    tokenizer, model = load_generation_model(generation_model)

    # Generate an answer
    answer = generate_answer(tokenizer, model, retrieved_chunks, query, max_new_tokens,temperature)

    # Display the answer
    print("===================GENERATED ANSWER===================")
    print(answer)

Loading: 60115-MBR.pdf
Loading: 60479-MBR.pdf
Loading: 59822_MBR.pdf
Loading: 60193-MBR.pdf
Loading: 60592-MBR.pdf
Loading: 59973-MBR.pdf
Loading: 60843-MBR.pdf
Loaded 7 PDFs from budgets/
Splitting pages from 60115-MBR.pdf into chunks...
Splitting pages from 60479-MBR.pdf into chunks...
Splitting pages from 59822_MBR.pdf into chunks...
Splitting pages from 60193-MBR.pdf into chunks...
Splitting pages from 60592-MBR.pdf into chunks...
Splitting pages from 59973-MBR.pdf into chunks...
Splitting pages from 60843-MBR.pdf into chunks...
Total chunks created: 312
Generating embeddings for chunks...
Vector store saved to vectorstore.faiss
Loading vector store from vectorstore.faiss...
Querying vector store for: 'What was the primary reason for the $309 billion increase in outlays by the Department of Education in fiscal year 2024?'
==========================Source Documents/Respective Pages=======================
 {'60193-MBR.pdf': [6, [7, 2, 6, 5, 4]], '60592-MBR.pdf': [7, [8, 6, 4, 5, 2]],

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████| 2/2 [00:03<00:00,  1.66s/it]


===================GENERATED ANSWER===================
The primary reason for the $309 billion increase in outlays by the Department of Education in fiscal year 2024 was because of the interest to loan balances in certain circumstances, and increased eligibility for the Public Service Loan Forgiveness program.

Reason: According to the passage, the increase in outlays by the Department of Education was primarily due to the interest to loan balances in certain circumstances, and increased eligibility for the Public Service Loan Forgiveness program. This is evident from the fact that the passage states that "no modifications have been recorded in the first seven months of fiscal year 2024" regarding the Department of Education's outlays, indicating that the increase in outlays is due to these two factors.

Therefore, the answer to the question is $309 billion.


# Model Evaluation

- It is hard to evalute the quaility of a RAG model since it is highly task-dependent. For the purpose of this notebook, we will experiment on the Montly Budget Reviews published by Congreational Budget Office. We developed a set of questions and answer along with where the information can be found. We will use file to see if our RAG pipeline is doing a decent job.
- In testing, we will also implement a simple reranking idea. We will count how many chunks is retrived per document based on the user query.
Of those chunks, we will retain chunks from the high-score documents. For example; if we have

          {'doc1.pdf': 6, 'doc2.pdf': 3, 'doc3.pdf': 1}

thresold = total_chunks * 0.2 = 10 * 0.2 = 2. Thus, we will consider the chunks from doc1 and doc2 and filter out doc3. Of course, there are all sort of other ideas to consider as well.

In [12]:
def evaluate_pipeline(test_data, vectorstore, tokenizer, model, max_chunks=20, max_new_tokens=400, ratio_to_keep=0.2):
    """
    Evaluate the retrieval and generation pipeline against test cases.
    
    Args:
        test_data (DataFrame): DataFrame containing Question, Answer, Reference, Page columns.
        vectorstore (FAISS): The vector store for retrieval.
        tokenizer: Tokenizer for the generation model.
        model: Generation model.
        max_chunks (int): Number of chunks to retrieve.
        max_new_tokens (int): Max tokens for generation.

    Returns:
        DataFrame: Results with comparison between generated and expected answers.
    """
    results = []

    for _, row in test_data.iterrows():
        question = row['Question']
        expected_answer = row['Answer']
        expected_reference = row['Reference']
        expected_page = row['Page']

        # Retrieve relevant chunks 
        retrieved_chunks = vectorstore.similarity_search(question, k=max_chunks)
        
        # Group chunks by their source metadata
        source_counts = Counter(chunk.metadata.get("source", "unknown") for chunk in retrieved_chunks)
        
        # Set a threshold to include multiple relevant sources
        threshold = max(1, int(len(retrieved_chunks) * ratio_to_keep))  # At least 20% or at least 1 chunk
        relevant_sources = [source for source, count in source_counts.items() if count >= threshold]
       
        
        # Filter chunks to include only those from relevant sources--> is this a good idea?
        filtered_chunks = [
            chunk for chunk in retrieved_chunks if chunk.metadata.get("source", "unknown") in relevant_sources
        ]
        
        # Combine filtered chunks as context, for now top-5 chunks and generate answer
        context = " ".join(chunk.page_content for chunk in filtered_chunks[:5])  # Adjust size if necessary
        generated_answer = generate_answer(tokenizer, model, filtered_chunks, question, max_new_tokens)
        
        # Extract metadata with a fancy helper function
        retrieved_references = [chunk.metadata.get("source", "unknown") for chunk in filtered_chunks]
        retrieved_pages = [chunk.metadata.get("page", "unknown") for chunk in filtered_chunks]
        formatted_references, formatted_pages = format_references_and_pages(retrieved_references,retrieved_pages)
        
        print(f"source information: {dict(source_counts)}")
        print("=====================================================")


        # we check if the retrived content contains the ground truth documents
        Is_In_Retrieved = "yes" if expected_reference in retrieved_references else "no"

        results.append({
            "Question": question,
            "Test Answer": expected_answer,
            "Model Answer": generated_answer,
            "Test Reference": expected_reference,
            "Model References": formatted_references,
            "Test Page": expected_page,
            "Model Pages": formatted_pages,
            "Is_In_Retrieved": Is_In_Retrieved
        })

    return pd.DataFrame(results)

In [13]:
if __name__ == "__main__":
    # pdf files and vector store foldr path
    data_path = "budgets/"
    vectorstore_path = "vectorstore.faiss"
    
    # simple logic to recreate or use the existing vector database
    UPDATE_VS = False
    
    # you can play with these as they directly effect the results
    chunk_size = 500
    chunk_overlap = 50
    
    # embedding and generation models
    embedding_model_name = 'FinLang/finance-embeddings-investopedia'
    #embedding_model_name = 'sentence-transformers/sentence-t5-base'
    generation_model = 'meta-llama/Llama-2-7b-chat-hf'
    
    # these are about how we manage the retrival and generation process
    max_chunks = 20        # max number of chunks retrived
    max_new_tokens = 300   # ouput tokens, lower if we need a short precise answer
    temperature = 0.7      # lower it if we dont need a creative generation
    ratio_to_keep = 0.2    # keep documents contributing to at least 20% chunks 

    # recrete and load the vector base if needed
    if UPDATE_VS:
        pdf_documents = load_pdfs_from_directory(data_path)
        chunks = convert_pdfs_chunks(pdf_documents, chunk_size, chunk_overlap)
        vectorstore = create_and_save_vector_store(chunks,embedding_model_name,vectorstore_path)
    vectorstore = load_vector_store(embedding_model_name,vectorstore_path)

    # Load generation model
    tokenizer, model = load_generation_model(generation_model)

    # Generate an answer
    test_data = pd.read_excel('cbo_questions.xlsx')

    # run evaluation
    results_df = evaluate_pipeline(test_data, vectorstore, tokenizer, model,
                               max_chunks, max_new_tokens, ratio_to_keep)
    results_df.to_csv('evaluation_results.csv',index=False)
   

Loading vector store from vectorstore.faiss...
source information: {'60843-MBR.pdf': 11, '60193-MBR.pdf': 1, '60115-MBR.pdf': 2, '60592-MBR.pdf': 3, '59973-MBR.pdf': 2, '59822_MBR.pdf': 1}
source information: {'60843-MBR.pdf': 7, '60115-MBR.pdf': 3, '59822_MBR.pdf': 2, '60592-MBR.pdf': 2, '59973-MBR.pdf': 3, '60479-MBR.pdf': 2, '60193-MBR.pdf': 1}


In [14]:
results_df

,Question,Test Answer,Model Answer,Test Reference,Model References,Test Page,Model Pages,Is_In_Retrieved
0,How does the percentage of GDP represented by ...,"In 2024, individual income tax receipts repres...","In 2024, individual income tax receipts repres...",60843-MBR.pdf,60843-MBR.pdf,3,"60843-MBR.pdf: [1, 1, 1, 3, 4, 4, 4, 4, 4, 5, 6]",yes
1,By how much did receipts from payroll taxes in...,Receipts from payroll taxes increased by $95 b...,"According to the text, receipts from payroll t...",60843-MBR.pdf,60843-MBR.pdf,4,"60843-MBR.pdf: [1, 1, 3, 4, 4, 4, 6]",yes


**Conclusion**

- Overally,our rag pipeline is not doing a bad job. We observed that if the documents are mostly composed of text, the pipeline works pretty well. We will soon deploy this pipeline to allow users to upload their own pdfs and communicate with them.
- In order to try out the code with a nice user interface, we prepared two different options with Gradio and Streamlit. You can use them as follows:
            
                                    streamlit run streamlit_UI.py
                                    python gradio_UI.py

This will promt a message where you can access the portals and interact with code above. Enjoy!
